# Dependencies

In [ ]:
from pinecone import Pinecone

from langchain_core.tools import tool
from pydantic.v1 import BaseModel, Field
from openai import OpenAI

from pydantic import BaseModel, Field, ValidationError
from typing import List, Dict, Optional


import os

from dotenv import load_dotenv
load_dotenv()

import os
import json
from crewai import Agent, Task, Crew, Process
from textwrap import dedent

from pydantic import BaseModel, Field
from typing import Optional

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")
PINECONE_QA_BASE_INDEX_NAME = os.getenv("PINECONE_QA_BASE_INDEX_NAME")
PINECONE_KNOWLEDGE_BASE_INDEX_NAME = os.getenv("PINECONE_KNOWLEDGE_BASE_INDEX_NAME")

# Client Initialisation
Now, we will be using the openAI client to get embeddings, as we would be needing in our model along with neccessary intialisations.

In [82]:
openAI_client = OpenAI(api_key=OPENAI_API_KEY)
embedding_model = openAI_client.embeddings
pc = Pinecone(api_key = PINECONE_API_KEY, environment = PINECONE_API_ENV)
index = pc.Index(PINECONE_QA_BASE_INDEX_NAME)

def get_embedding(text) :
    """
        Function to convert the text string into embeddings using text-embedding-3-small from OpenAI
    
        Args:
            text : A string which will contain either the text chunk or the user query
            
        Returns:
            vector : A vector of 1536 dimensions
    """
    
    try:
        response = embedding_model.create(
            input=text,
            model="text-embedding-3-small"
        )
        
        return response.data[0].embedding   
    
    except Exception as e:
        raise Exception(str(e))
    

Now, to use the above, we need to create these functions as a collective tool, which can be used by an agent to query the vector DB.

# Tool Creation
Now, we would have to create tools for this purpose, as asynchronus functions which await till the vector DB is loaded and then query the vector DB, evnetually returning the relevant texts. As the admin id will be a fixed one, we will be directly referring it from the environment.

In [83]:
@tool
def answer_query_qa(query):
    """
        Find relevant documents for a given query and answer the query using the relevant documents.
        
        Args:
        - query (str): The search query will be in string format which will be passed straight to the LLM to find the relevant documents.
        
        Returns:
            The relevant documents from the database, given the query
    """
    query_vector = get_embedding(query)
    
    results = index.query(
        vector = query_vector,
        top_k = 10,
        include_values = False,
        include_metadata = True,
        filter={
            "ADMIN_ID" : "d359d72b-40e8-4e9f-b567-62d77f273113",
        }
    )
    
    relevant_pairs = []
    for record in results['matches']:
        pair = {}
        pair['id'] = record['metadata']['QA_ID']
        pair['score'] = record['score']
        pair['question'] = record['metadata']['question']
        pair['answer'] = record['metadata']['answer']
        relevant_pairs.append(pair)
    
    if not relevant_pairs : 
        return [{
            'id' : 'NA',
            'score' : -1.00,
            'question' : query,
            'answer' : "We could not find any relevant documents for the given query"
        }]
        
    return relevant_pairs
    

# Agents
Now, we will have to make the tools for the agent and make it ready to be used. We will be using three agents : 

1. Researcher
2. Writer
3. Editor

To give recommendations to any previosuly solved query before.

In [84]:
researcher = Agent(
    role = "researcher",
    goal = "To research and find the relevant Q&As from the past to the query {query}",
    backstory = """
            You work as a researcher in an edtech startup 100xDevs, where your primary job is to research and give all the 
            relevant documents/chunks of previously solved Q&As, where your task is to collate those references which will 
            aid to solve the query : {query}. Your work will be directly passed to the writer, hence make sure you provide 
            references only which you can quote with the relevant thread ID (you will find it as id). You have been given a tool 
            to fetch the most relevant documents, which will give you a list of all the relevant documents, which 
            consist of question, answer, id (this will be that thread's id on discord) and a score for you to evaluate on which is most relevant.
            In case you get irrelevant documents, the tool has handled that too and you just need to pass it on to the writer.
    """,
    verbose = True,
    allow_delegation = False,
    tools = [answer_query_qa]
)

In [85]:
writer = Agent(
    role = "writer",
    goal = "To structure and answer the query : {query} given the relevant documents from the researcher",
    backstory = """
            You are a recommendation analyst working at an edTech startup 100xDevs, where your job is to recommend the students 
            on whether their query or something similar has been answered before, given the query : {query}. \n
            Your work is dependent on the work of the researcher, who will bring you all the relevant information and your job is to collate it into a proper 
            explanation of what the question was and how it was solved (make sure to keep the entire solution/answer intact) along with it's reference.
            You will be writing your recommendation with acknowledging and referencing each factual statement as given by the researcher, 
            hence it is important you MUST give the thread ID to your editor, should you find a relevant answer.\n
            In case you find the research to be irrelevant, you can just let your editor know that there is no such query or solution solved 
            previously to the said developer. There is no need to solve it yourself, only solve if your have the relevant research.
    """,
    allow_delegation = False,
    verbose = True,
)

In [98]:
editor = Agent(
    role = "editor",
    goal = "To edit the writer's recommendation to answer the query : {query} and interact with new student queries",
    backstory = """
            You are the senior editor at an edTech startup 100xDevs, where your job is to handle and give recommendation messages to the
            new queries created by students on our discord channel. You work is dependent on the writer's content piece, as that would have information
            on the most relevant answer and reference they could find to the query : {query}. In case the writer is unable to produce any results or 
            denies any relevant findings to base their content on, you do not need to pass anything to the user as handled in your task.\n
    """,
    allow_delegation = False,
    verbose = True,
)

# Tasks

In [87]:
class ResearcherDocuments(BaseModel) : 
    id : str = Field(...,description="The discord ID of the thread, from which this document is quoted")
    score : float = Field(...,description="The score implying the relvance of this document to the given query, with 0 being the lowest and 1 being the highest, and -1 being irrelevant")
    question : str = Field(...,description="The question which was asked in that document")
    answer : str = Field(...,description="The answer which was provided earlier in that document")

In [88]:
class ResearcherOutput(BaseModel) : 
    research : List[ResearcherDocuments] = Field(...,description="The discord ID of the thread, from which this document is quoted")

In [89]:
research = Task(
    description="To research and find the most relevant documents and references to the given query : {query}",
    expected_output="""A list consisting of all the relevant documents, where each document (JSON) follows the structure : 
        id : The thread ID of that conversation on discord
        score : The relevance of the said document, with 0 being lowest and 1 being highest and -1 being irrelvant
        question : The primary question which was asked in the thread
        answer : The answer/solution which was provided to that thread
    """,
    agent=researcher,
    parameters={"query": "str"},
    output_pydantic= ResearcherOutput
)

In [90]:
class WriterOutput(BaseModel) : 
    id : Optional[str] = Field(...,description="The discord ID of the thread, from which this document is quoted")
    content : str = Field(...,description="The content written by the writer to answer the query, given the research")

In [91]:
write = Task(
    description="To write content providing the solution to the query : {query}, given the research done by researcher",
    expected_output="""A object consisting of all the content written by the writer, where it follows the structure : 
        id : The thread ID of that conversation on discord, only if the writer is able to find relevant content (This is an OPTIONAL key)
        content : The content piece written by the writer, if they are supported by the research team
    """,
    agent=writer,
    output_pydantic=WriterOutput
)

In [92]:
class EditorOutput(BaseModel) : 
    status : int = Field(...,description="The status code of the answer given by the agent")
    relevance : bool = Field(...,description="Whether the team was able to find a potentially relevant solution or not")
    solution : Optional[str] = Field(...,description="The final output given by the team, given that they are able to find a relevant solution")
    source : Optional[str] = Field(...,description="The reference to the answer, which is the exact thread ID on discord, given that they are able to find a relevant solution ")

In [93]:
edit = Task(
    description="To edit and give the status of work and final output as a recommendation to the student, if any content given by writer",
    expected_output="""A object consisting of all the final output given by the editor, with the structure of it as follows : 
        status : The status code of the team's findings. If any content given by writer, then 200 else 404.
        relevance : A boolean value indicating whether the team was able to complete the task or not
        solution : The final edited solution from the editor, if none then give the string "NA"
        source : The exact thread ID on discord from which this solution is quoted, if none then give the string "NA"
    """,
    agent=editor,
    output_pydantic=EditorOutput
)

# Crew

In [94]:
crew = Crew(
    agents=[researcher,writer,editor], 
    tasks=[research,write,edit],
    memory=True,
    verbose=True
)

Overriding of current TracerProvider is not allowed


# Execution

In [76]:
query = "I was having issues in my NEXTJS project, as I tried to animate the dropdown menu, I got a hydration error."
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : I was having issues in my NEXTJS project, as I tried to animate the dropdown menu, I got a hydration error.


# Agent: researcher
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"I was having issues in my NEXTJS project, as I tried to animate the dropdown menu, I got a hydration error.\"}"
## Tool Output: 

    

 User Query : I was having issues in my NEXTJS project, as I tried to animate the dropdown menu, I got a hydration error.
    

 Documents : [{'id': '9cef8bde-c6a7-4758-a736-7d2097c0b38a', 'score': 0.728818119, 'question': 'I encountered a hydration error in my Next.js project while trying to animate a dropdown menu, and after undoing changes, my styles disappeared. How can I resolve this?', 'answer': "To resolve the hydration error and styling issues in your Next.js project, first restart the server and inspect the styles using the browser's inspect e

{'status': 200,
 'relevance': True,
 'solution': "To resolve the hydration error you are encountering in your Next.js project while trying to animate the dropdown menu, follow these steps: \n\n1. **Restart Your Server**: Sometimes, simply restarting the Next.js server can resolve unexpected issues.\n2. **Inspect Styles**: Use the browser's inspect element tool to verify that your styles (e.g., Tailwind CSS) are being loaded correctly. This can help you identify if the necessary CSS is being applied properly.\n3. **Check Tailwind CSS Configuration**: If the styles are present in the inspection tab but not applied, it may indicate a configuration issue with Tailwind CSS. Review the Tailwind CSS documentation to ensure your setup is correct. This often involves:\n   - Verifying your `tailwind.config.js` file for correct configuration.\n   - Ensuring that your Tailwind classes are being generated and correctly applied to the elements. \n\nBy following these steps, you should be able to ide

In [77]:
query = "I encountered an issue with CORS in my application, and it was resolved by removing a 'return' statement. Why did this happen?"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : I encountered an issue with CORS in my application, and it was resolved by removing a 'return' statement. Why did this happen?


# Agent: researcher
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"I encountered an issue with CORS in my application, and it was resolved by removing a 'return' statement. Why did this happen?\"}"
## Tool Output: 

    

 User Query : I encountered an issue with CORS in my application, and it was resolved by removing a 'return' statement. Why did this happen?
    

 Documents : [{'id': '2cef8bde-c6a7-4758-a836-7d2097c0b38a', 'score': 0.828187525, 'question': "I encountered an issue with CORS in my application, and it was resolved by removing a 'return' statement. Can someone explain why this happened?", 'answer': 'The issue with CORS was resolved by adjusting the middleware configuration. Initially, there was a suggestion to use `a

{'status': 200,
 'relevance': True,
 'solution': 'This query may have been solved before. The issue with CORS was resolved by adjusting the middleware configuration. Initially, there was a suggestion to use `app.use(cors())` with an empty string, which was corrected by adding `origin: *` to allow all origins. Further advice was given to specify the frontend origin using `app.use(cors({ origin: process.env.FRONTEND_URL || "http://localhost:5173", methods: [\'GET\', \'POST\', \'PUT\', \'DELETE\'], credentials: true }));`. The problem was resolved when a \'return\' statement was removed, which likely interfered with the middleware execution flow. The \'return\' statement might have prematurely terminated the function, preventing the CORS configuration from being applied correctly. Rebuilding the server and ensuring the correct CORS setup helped in resolving the issue.',
 'source': '2cef8bde-c6a7-4758-a836-7d2097c0b38a'}

In [95]:
query = "How do JWT tokens work?"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : How do JWT tokens work?


# Agent: researcher
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"How do JWT tokens work?\"}"
## Tool Output: 
[{'id': '2cef8bde-c6a7-4758-a836-7d2097c0b38a', 'score': 0.223142341, 'question': "I encountered an issue with CORS in my application, and it was resolved by removing a 'return' statement. Can someone explain why this happened?", 'answer': 'The issue with CORS was resolved by adjusting the middleware configuration. Initially, there was a suggestion to use `app.use(cors())` with an empty string, which was corrected by adding `origin: *` to allow all origins. Further advice was given to specify the frontend origin using `app.use(cors({ origin: process.env.FRONTEND_URL || "http://localhost:5173", methods: [\'GET\', \'POST\', \'PUT\', \'DELETE\'], credentials: true }));`. The problem was resolved when a \'return\' statement was

{'status': 404, 'relevance': False, 'solution': 'NA', 'source': 'NA'}

In [96]:
query = "ughhh I am geting 'pnpm run dev' error in my project, somebody help??"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : ughhh I am geting 'pnpm run dev' error in my project, somebody help??


# Agent: researcher
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"ughhh I am geting 'pnpm run dev' error in my project, somebody help??\"}"
## Tool Output: 
[{'id': '2cef8bde-c6a7-4758-a736-7d2097c0b38a', 'score': 0.600647688, 'question': "I am getting an error while executing the code for the Excalidraw Project using 'pnpm run dev'. Can someone help me solve this error?", 'answer': 'The issue was resolved by checking the configuration of the http-server in the package.json file and ensuring the use of the latest version of Node.js. The teaching assistants suggested configuring the http-backend app and updating Node.js, which helped in resolving the error.'}, {'id': '9cef8bde-c6a7-4758-a736-7d2097c0b38a', 'score': 0.314720541, 'question': 'I encountered a hydration error in my Next.js pr

{'status': 200,
 'relevance': True,
 'solution': "The error encountered while using 'pnpm run dev' in your project appears related to the configuration of the http-server within your package.json file. To resolve this error, it is suggested to check and verify the configuration settings there. Additionally, it is essential to ensure that you are using the latest version of Node.js, as outdated versions may cause compatibility issues. The teaching assistants recommend configuring the http-backend application and updating Node.js, which was effective in resolving the error. After making these adjustments, you should be able to run your development server without issues.",
 'source': '2cef8bde-c6a7-4758-a736-7d2097c0b38a'}

In [97]:
query = "yaar mujhe app banana hai where dashboard loads and UI updates happen immediately, backend sync ke saath"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : yaar mujhe app banana hai where dashboard loads and UI updates happen immediately, backend sync ke saath


# Agent: researcher
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"yaar mujhe app banana hai where dashboard loads and UI updates happen immediately, backend sync ke saath\"}"
## Tool Output: 
[{'id': '2cef8bde-c6a7-4758-b736-7d2097c0b38a', 'score': 0.49639377, 'question': 'How can I build a second brain app using Recoil to fetch all data when the /dashboard component loads and ensure the UI updates immediately with backend synchronization?', 'answer': 'To build a second brain app using Recoil, you should use an atom to store the state and combine it with a useEffect hook to fetch initial data when the /dashboard component loads. This setup ensures that the UI updates immediately by modifying the atom after every operation such as create, edit, or delete

{'status': 200,
 'relevance': True,
 'solution': 'To build an app where the dashboard loads, and the UI updates immediately with backend synchronization, you can utilize Recoil for state management. Here’s how to proceed: First, set up an atom in Recoil to store the state of your dashboard. Then, leverage the useEffect hook to fetch the initial data when the /dashboard component loads. This combination ensures that any modifications you make—like create, edit, or delete—will immediately update this atom, thus reflecting changes in the UI instantaneously. The useEffect will be responsible for fetching data from your backend, and by maintaining the state within the atom, you will achieve real-time updates and synchronization with the backend effectively.',
 'source': '2cef8bde-c6a7-4758-b736-7d2097c0b38a'}

With this, we now have an agent which will be answering the core question asked by the user, based on the Q&A knowledge base. This is suppose to work as a FASTAPI endpoint whenever the user creates a new thread, all previous threads can be inspected for the same.